# Boiler

In [1]:
import os

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from glob import glob

import dask
from dask.diagnostics import ProgressBar
dask.config.set(**{'array.slicing.split_large_chunks': True})

In [2]:
# import dask.distributed as dd
# import tempfile

# tempdir = tempfile.TemporaryDirectory("dask-worker-space")
# dd.Client(local_directory=tempdir.name, memory_limit='16gb')

# from distributed import Client
# client = Client()



# from dask_jobqueue import PBSCluster
# # One node on Gadi has 48 cores - try and use up a full core before going to multiple nodes (jobs)

# walltime = '00:10:00'
# cores = 48
# memory = str(cores * 4) + 'GB'

# cluster = PBSCluster(walltime=str(walltime), cores=cores, memory=str(memory), processes=cores,
#                      job_extra_directives=['-q normal',
#                                            '-P w42',
#                                            '-l ncpus='+str(cores),
#                                            '-l mem='+str(memory),
#                                            '-l storage=gdata/w42+gdata/rt52'],
#                      local_directory='$TMPDIR',
#                      job_directives_skip=["select"])
# cluster.scale(jobs=1)
# client = Client(cluster)

# Loading Data

In [4]:
MJO_DIR = '/g/data/w40/ab2313/rainfall_2020/mjo_index'
AGCD_DIR = '/g/data/zv2/agcd/v1/precip/calib/r005/01day'

In [9]:
files = glob(os.path.join(AGCD_DIR, '*.nc'))
files = np.sort([f for f in files if int(f.split('_')[-1].split('.')[0]) >= 1974]) # Just files with year > 1974
files[:5]

array(['/g/data/zv2/agcd/v1/precip/calib/r005/01day/agcd_v1_precip_calib_r005_daily_1974.nc',
       '/g/data/zv2/agcd/v1/precip/calib/r005/01day/agcd_v1_precip_calib_r005_daily_1975.nc',
       '/g/data/zv2/agcd/v1/precip/calib/r005/01day/agcd_v1_precip_calib_r005_daily_1976.nc',
       '/g/data/zv2/agcd/v1/precip/calib/r005/01day/agcd_v1_precip_calib_r005_daily_1977.nc',
       '/g/data/zv2/agcd/v1/precip/calib/r005/01day/agcd_v1_precip_calib_r005_daily_1978.nc'],
      dtype='<U83')

In [10]:
# 3https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f6475_9317_5747_6204
agcd_ds = xr.open_mfdataset(files).precip
agcd_ds = agcd_ds.chunk({'time':-1, 'lat':int(691/20), 'lon':(int(886/16))}) # Chunking in time
agcd_ds['time'] = agcd_ds.time.values - pd.to_timedelta('9H') # Adjust time to match rmm
agcd_ds = agcd_ds.where(agcd_ds > 1) # Remove rainfall less than 1mm
agcd_ds

,Array,Chunk
Bytes,38.67 GiB,120.93 MiB
Shape,"(16953, 691, 886)","(16953, 34, 55)"
Count,3908 Tasks,357 Chunks
Type,float32,numpy.ndarray


# Climatology

In [7]:
agcd_monthly_sum_climatology_ds = agcd_ds.groupby('time.month').sum(dim='time')
agcd_monthly_count_climatology_ds = agcd_ds.groupby('time.month').count(dim='time')

agcd_rprd_clim = agcd_monthly_sum_climatology_ds/agcd_monthly_count_climatology_ds

In [7]:
with ProgressBar():
    agcd_rprd_clim = agcd_rprd_clim.compute()

[                                        ] | 0% Completed |  2min  1.7s


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.01/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3524, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/local/w40/ab2313/tmp/ipykernel_1302256/3063088907.py", line 2, in <cell line: 1>
    agcd_rprd_clim = agcd_rprd_clim.compute()
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.01/lib/python3.9/site-packages/xarray/core/dataarray.py", line 947, in compute
    return new.load(**kwargs)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.01/lib/python3.9/site-packages/xarray/core/dataarray.py", line 921, in load
    ds = self._to_temp_dataset().load(**kwargs)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.01/lib/python3.9/site-packages/xarray/core/dataset.py", line 861, in load
    evaluated_data = da.compute(*lazy_data.values(), **kwargs)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.01/lib/pyth

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.01/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3524, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/local/w40/ab2313/tmp/ipykernel_1302256/3063088907.py", line 2, in <cell line: 1>
    agcd_rprd_clim = agcd_rprd_clim.compute()
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.01/lib/python3.9/site-packages/xarray/core/dataarray.py", line 947, in compute
    return new.load(**kwargs)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.01/lib/python3.9/site-packages/xarray/core/dataarray.py", line 921, in load
    ds = self._to_temp_dataset().load(**kwargs)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.01/lib/python3.9/site-packages/xarray/core/dataset.py", line 861, in load
    evaluated_data = da.compute(*lazy_data.values(), **kwargs)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.01/lib/pyth

TypeError: object of type 'NoneType' has no len()